## get Cause efffect phrase from the sentence.

In [1]:
#load CE semeval dataset
import pandas as pd
sem_CE_test= pd.read_csv('data/in/KNN_RAG/ADE_CE_extract_TEST.csv')

pm_DB = sem_CE_test

#start#comparison with DEBERT paper table
pm_DB = pd.read_csv('data/in/KNN_RAG/sem_CE_test_cleaned.csv')
pm_DB = pm_DB[107:]
pm_DB.reset_index(drop=True, inplace=True)

##end

pm_DB = pm_DB.rename(columns={'cause': 'cause_t', 'effect': 'effect_t'})
pm_DB

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sentence,cause_t,effect_t,embedings
0,107,158,158,158,158,"Finally, Slone's fear of AIDS and the mental...",stab,fear,"[-0.00929092988371849, -0.00021748989820480347..."
1,108,159,159,159,159,The suffering caused by the embargo is som...,embargo,suffering,"[-0.009257652796804905, -0.031496670097112656,..."
2,109,160,160,160,160,Sorace was unaware that her anger was caused...,abuse,anger,"[0.01198310311883688, -0.010999979451298714, 0..."
3,110,161,161,161,161,The idea of Roman expansion into North Africa ...,power,fear,"[0.017029738053679466, -0.010211342945694923, ..."
4,111,162,162,162,162,Tides are caused by the gravitational pull ...,gravitational pull,Tides,"[0.010173842310905457, -0.015142759308218956, ..."
...,...,...,...,...,...,...,...,...,...
137,244,321,321,321,321,"The winds are caused by moisture , originat...",moisture,winds,"[-0.0030969225335866213, -0.015087406150996685..."
138,245,322,322,322,322,The drag caused by the Earth's atmosphere ...,atmosphere,drag,"[-0.00888124480843544, -0.003990810364484787, ..."
139,246,323,323,323,323,Most of the flooding has been caused by the ...,clogging,flooding,"[-0.01667359657585621, 0.0030442047864198685, ..."
140,247,324,324,324,324,The dips are caused by the thermal diffuse o...,y-rays,dips,"[-0.018423471599817276, -0.0059008412063121796..."


In [2]:
#load causality input sentence.
#input_sentence_list = filtered_rows['sentence'].to_list()
#gpt to extract cause effect and a causality phrase.
#: few shot: example1: , causality phrase: .

In [3]:
from openai import OpenAI
client = OpenAI(api_key='enter your API key here..')

###model#gpt-3.5-turbo

#+ fewshot_examples
def C_E_CP_Identifier(input_sentence, fewshot_examples):
    
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      response_format= {"type": "json_object"},
      messages=[
        {
          "role": "system",
          "content": '''
  
          You will be given a sentence that shows a causality relation. 
          Your task is to identify the cause phrase, effect phrase, and causality phrase from
          that sentence. Below is the hints and rules for the task.

          Hints: In sentences showing a causality relation, you can identify a pair of phrases
          where one phrase causes the other (effect). There will also be a phrase between the
          cause and effect phrases that indicates the causality relation; this is the 
          causality phrase. And in most sentences causality phrase can be seen between the 
          cause and effect phrase.
           

          Your output should be in JSON format with keys cause, effect, and causality_phrase:
          {"cause": "", "effect": "", "causality_phrase": ""}.
          
          Rule 1: If you are not able to find the cause phrase, effect phrase, and causality phrase 
          from the given sentence you can output empty string for the keys in the json format 
          and do not guess the answer. 
          
          Rule 2: Ensure that there are no overlapping phrases between the cause phrase, effect phrase,
          and causality phrase. For instance, consider the input sentence "Fever is caused by virus." 
          Your identified output without overlap should be:
          {"cause": "virus", "effect": "fever", "causality_phrase": "caused by"}
          Your output should not contain overlap, as shown below:
          {"cause": "by virus", "effect": "fever is caused", "causality_phrase": "caused by"}
          This means that each phrase (cause, effect, and causality phrase) should be distinct 
          and not overlap with each other.
                   
          Below are 10 example sentences that show causality relations, with their cause and
          effect phrases delimited between the XML tags <cause>, </cause>, <effect>, and 
          </effect>. The corresponding causality phrases are also provided.
          

                   
          ''' + fewshot_examples
            
        },
        {
          "role": "user",
          "content": input_sentence 
        }
      ],
      temperature=0.0,
      seed = 1,
      max_tokens=200,
      top_p=1
    )
    
    return response.choices[0].message.content


In [4]:
def get_fewshotexamples_C_E_CP(fewshot_index):

    
    #fewshot_index = [1, 5, 8, 10, 30, 100, 102, 105, 109, 120]
    
    
    train_sentence = semeval_data['sentence'].to_list()
    train_causalityPhrase = semeval_data['causality_phrase'].to_list()

    fewshot_example_sentence = [train_sentence[i] for i in fewshot_index]
    fewshot_example_causalityPhrase = [train_causalityPhrase[i] for i in fewshot_index]


    import json
    #fewshot_examples = {f'example {i + 1}': sentence for i, sentence in enumerate(fewshot_examples)}
    fewshot_examples = {f'example {i + 1}': {'sentence': sentence, 'causalityPhrase': fewshot_example_causalityPhrase[i]} 
                    for i, sentence in enumerate(fewshot_example_sentence)}
    
    # Assuming json_data is already defined
    fewshot_examples = json.dumps(fewshot_examples)
    #print(fewshot_examples)
    
    return fewshot_examples

In [5]:
# semeval
filtered_rows = pm_DB

In [6]:
#load semeval sentence and patterns.
semeval_data = pd.read_csv('data/in/RAG/comb_SCITEsemADE_CausalityPattern.csv')# 

semeval_data = semeval_data.dropna(subset=['causality_phrase'])
len(semeval_data)

#semeval_data = pd.read_csv('data/in/semeval_CE_pattern.csv')# data/in/ADE/ADE_CE_pattern.csv
causality_phrase = semeval_data['causality_phrase'].to_list()
len(causality_phrase)

2365

In [7]:
#pattern matcher function.
import random
from fuzzywuzzy import fuzz

def get_fewshot_examples_index(input_sentence) :

    #get few shot example index using Leiveinstein ratio.

    fewshot_index = []
    for ind in range (len(causality_phrase)):
        p = causality_phrase[ind]
        #print('ind :', ind)
        window_size = len(p)

        # Slide through input_sentence with a window size 
        for i in range(len(input_sentence) - window_size + 1):
            window = input_sentence[i:i + window_size]
            score = fuzz.ratio(window, p)
            if score >90:
                fewshot_index.append(ind)
                #print('causality phrase: ',p, 'index is', ind)
                #print(f"Window '{window}' - LS: {score}")

    #print('fewshot_index', fewshot_index)
    fewshot_index = list(set(fewshot_index))
    #print('fewshot_index', fewshot_index)
    
    if len(fewshot_index)>10:
        fewshot_index = random.sample(fewshot_index, 10)
        
    
    return fewshot_index
    

#get all sentences and save as json example1

/home/tnaduvil/.conda/envs/SCITE/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [8]:
import json

cause_list = []
effect_list = []
causality_phrase_list = []
fewshotfound_inputsentence_index=[]
for i in range (len(filtered_rows)):
    input_sentence = filtered_rows['sentence'][i]
    #input_sentence = 'fever is caused by virus'
    
    #print('input_sentence', input_sentence)
    
    # few shot examples augmentation.
    fewshot_index = get_fewshot_examples_index(input_sentence)#########FEWSHOT
    
    ##
    if len(fewshot_index) == 0:
        fewshot_index = [k for k in range(10)]
        fewshot_examples =  get_fewshotexamples_C_E_CP(fewshot_index) 
    else:
        fewshotfound_inputsentence_index.append(i)
        
        #fewshot_index = [k for k in range(10)]###############################RANDOMSHOT-ZERO
        fewshot_examples =  get_fewshotexamples_C_E_CP(fewshot_index) 
        
    ##
    print('fewshot_index :', fewshot_index)
    print('fewshot_examples', fewshot_examples)

    #fewshot_examples = get_fewshotexamples_C_E_CP(fewshot_index) 
    
    #print('fewshotfound_inputsentence_index', fewshotfound_inputsentence_index)
    
    # output generation
    out = C_E_CP_Identifier(input_sentence, fewshot_examples)
    
    print('out: ', out)
    
    json_out = json.loads(out)
    cause = json_out['cause']
    effect = json_out['effect']
    causality_phrases = json_out['causality_phrase']
    
    
    cause_list.append(cause)
    effect_list.append(effect)
    causality_phrase_list.append(causality_phrases)
    
    #print('input_sentence: ', input_sentence )
    #print('fewshot_examples: ', fewshot_examples )
    #print('cause: ', cause)
    #print('effect: ', effect)
    #print('causality_phrase: ', causality_phrases)
    
    
    print(i)
    #if i==0:
        #break;
    #print(causality_identifier)
    
    

fewshot_index : [479, 1531, 1026, 1241, 1506, 317, 1508, 1511, 992, 997]
fewshot_examples {"example 1": {"sentence": "\"It was also felt necessary at this time to draw up a similar document, dealing not with the prevention of marine <effect>pollution</effect> by <cause>dumping</cause>, but instead with the prevention of marine pollution by discharges of dangerous substances from land-based sources, watercourses or pipelines.\"", "causalityPhrase": "by"}, "example 2": {"sentence": "A case of <effect>heatstroke</effect> is reported in a 32-year-old man diagnosed with schizophrenia and on <cause>clozapine</cause> monotherapy.", "causalityPhrase": "on"}, "example 3": {"sentence": "\"There is relatively little <effect>discomfort</effect> from this <cause>surgery</cause> and most individuals rarely take more than Tylenol for their discomfort.\"", "causalityPhrase": "from this"}, "example 4": {"sentence": "The most commonly recognized toxic effect of <cause>ethambutol</cause> is <effect>optic

out:  {"cause": "fear and jealousy", "effect": "great economic power of Carthage", "causality_phrase": "caused by"}
3
fewshot_index : [1530, 323, 1513, 520, 592, 1512, 523, 1531, 1515, 514]
fewshot_examples {"example 1": {"sentence": "<effect>Scleromyxedema</effect> in a patient with multiple sclerosis and monoclonal gammopathy on <cause>interferon beta-1a</cause>.", "causalityPhrase": "on"}, "example 2": {"sentence": "\"Many <effect>shoulder problems</effect> are caused by the <cause>breakdown</cause> of soft tissues in the shoulder region.\"", "causalityPhrase": "are caused by"}, "example 3": {"sentence": "PURPOSE: We report an unusual <effect>paradoxical effect</effect> of <cause>brimonidine</cause>.", "causalityPhrase": "of"}, "example 4": {"sentence": "\"<cause>Overpopulation</cause> and poverty cause <effect>tension</effect> not only within communities and countries, but throughout the world.\"", "causalityPhrase": "cause"}, "example 5": {"sentence": "\"The <effect>suffering</eff

out:  {"cause": "eating the tomatoes right off the vine", "effect": "joy", "causality_phrase": "from"}
7
fewshot_index : [999, 480, 1531, 1000, 2081, 2337, 1513, 1507, 1528, 2088]
fewshot_examples {"example 1": {"sentence": "\"The <effect>benefits</effect> from <cause>inflation</cause> which Professor Mulligan cites (e.g., bailing out underwater mortgagers) are better understood as a government sponsored transfer of wealth from savers to speculators.\"", "causalityPhrase": "from"}, "example 2": {"sentence": "CONCLUSION: We have introduced a case of <effect>anaphylaxis </effect>by <cause>calcitonin</cause> that suggest an IgE mediated hypersensitivity reaction.", "causalityPhrase": "by"}, "example 3": {"sentence": "A case of <effect>heatstroke</effect> is reported in a 32-year-old man diagnosed with schizophrenia and on <cause>clozapine</cause> monotherapy.", "causalityPhrase": "on"}, "example 4": {"sentence": "\"The risks to housing and general corporate <effect>profits</effect> from <

out:  {"cause": "tsunami waves", "effect": "devastation", "causality_phrase": "caused by"}
11
fewshot_index : [221, 212, 218, 215, 1136, 1242, 1533, 123, 1241, 118]
fewshot_examples {"example 1": {"sentence": "\"The film also features documentary like segments about fishing and about the <effect>evacuation</effect> after the <cause>eruption</cause> of the volcano.\"", "causalityPhrase": "after the"}, "example 2": {"sentence": "\"The patient experiences displaced emotional and financial <effect>deprivation</effect> as well as anger after the <cause>divorce</cause>.\"", "causalityPhrase": "after the"}, "example 3": {"sentence": "\"The log truck caught <effect>fire</effect> after the <cause>collision</cause>, but the 35-year-old driver escaped before the cab became engulfed in flames.\"", "causalityPhrase": "after the"}, "example 4": {"sentence": "\"Stella Liebeck is the woman who sued McDonalds after she suffered a serious <effect>burn</effect> after the <cause>spilling</cause> of a cup 

out:  {"cause": "shipwreck", "effect": "pollution", "causality_phrase": "was caused by"}
15
fewshot_index : [2338, 2340, 1529, 1512, 1506, 481, 517, 598, 480, 1241]
fewshot_examples {"example 1": {"sentence": "We conclude <effect>peripheral neuropathy</effect> with <cause>5-FU</cause> is rare.", "causalityPhrase": "with"}, "example 2": {"sentence": "Rapid resolution of topiramate-induced <effect>angle-closure glaucoma</effect> with methylprednisolone and <cause>mannitol</cause>.", "causalityPhrase": "with"}, "example 3": {"sentence": "We present an AIDS patient with severe and prolonged <effect>lactic acidosis</effect> on stavudine and <cause>lamivudine</cause>.", "causalityPhrase": "on"}, "example 4": {"sentence": "A <effect>paradoxical ocular effect</effect> of <cause>brimonidine</cause>.", "causalityPhrase": "of"}, "example 5": {"sentence": "\"The basic <effect>indicator</effect> of the health <cause>condition</cause> of trees, stands and forests is tree defoliation.\"", "causalityP

...

out:  {"cause": "death of my friend", "effect": "fear", "causality_phrase": "triggered by"}
123
fewshot_index : [590, 2094, 518, 1020, 520, 2091, 1528, 557, 279, 481]
fewshot_examples {"example 1": {"sentence": "\"The <effect>burst</effect> has been caused by water hammer <cause>pressure</cause>.\"", "causalityPhrase": "caused by"}, "example 2": {"sentence": "After nine previous uncomplicated cycles she developed severe <effect>anaphylaxis</effect> to <cause>cisplatin</cause>.", "causalityPhrase": "to"}, "example 3": {"sentence": "\"Pressures occur throughout life and those <cause>pressures</cause> cause <effect>stress</effect>.\"", "causalityPhrase": "cause"}, "example 4": {"sentence": "\"An alert victim with a broken neck or severely torn ligament has enough <effect>discomfort</effect> from the <cause>injury</cause> and muscle spasm to force him to hold his neck still.\"", "causalityPhrase": "from the"}, "example 5": {"sentence": "\"<cause>Overpopulation</cause> and poverty cause <ef

out:  {"cause": "environmentally-induced collapse of fish catches from the Black Sea", "effect": "decline", "causality_phrase": "has been caused by"}
127
fewshot_index : [2088, 2086, 2087, 992, 996, 2094, 1001, 998, 999, 997]
fewshot_examples {"example 1": {"sentence": "<effect>Systemic allergic contact dermatitis</effect> to <cause>8-methoxypsoralen</cause> (8-MOP).", "causalityPhrase": "to"}, "example 2": {"sentence": "<effect>Anaphylaxis</effect> to <cause>calcitonin</cause>.", "causalityPhrase": "to"}, "example 3": {"sentence": "<effect>Lichenoid drug eruption</effect> to <cause>salsalate</cause>.", "causalityPhrase": "to"}, "example 4": {"sentence": "\"He had chest pains and <effect>headaches</effect> from <cause>mold</cause> in the bedrooms.\"", "causalityPhrase": "from"}, "example 5": {"sentence": "\"I too, get a <effect>headache</effect> from <cause>wine</cause>, and was always told that it was the sulfites.\"", "causalityPhrase": "from"}, "example 6": {"sentence": "After nine 

out:  {"cause": "fatigue and allergies", "effect": "headaches", "causality_phrase": "from"}
131
fewshot_index : [1097, 592, 589, 1510, 593, 1508, 477, 1531, 519, 595]
fewshot_examples {"example 1": {"sentence": "\"First, the H5N1 <cause>virus</cause> has caused by far the greatest number of human cases of very severe <effect>disease</effect> and the greatest number of deaths.\"", "causalityPhrase": "has caused"}, "example 2": {"sentence": "\"The <effect>suffering</effect> caused by the <cause>terrorists</cause> is the real torture.\"", "causalityPhrase": "caused by"}, "example 3": {"sentence": "\"<cause>Fatigue</cause> <effect>corrosion</effect> and stress corrosion are similar, as both are caused by external stresses applied to the pipe and occur inside of the pipe.\"", "causalityPhrase": "caused by"}, "example 4": {"sentence": "<effect>Neurotoxicity</effect> of <cause>valacyclovir</cause> in peritoneal dialysis: a pharmacokinetic study.", "causalityPhrase": "of"}, "example 5": {"sent

out:  {"cause": "illegal embedded code tags", "effect": "error", "causality_phrase": "was caused by"}
135
fewshot_index : [478, 518, 1506, 1527, 514, 481, 213, 219, 124, 1514]
fewshot_examples {"example 1": {"sentence": "\"Before doing for his hair <effect>reduction</effect> by laser <cause>treatment</cause>, he first consulted with a dermatologist.\"", "causalityPhrase": "by"}, "example 2": {"sentence": "\"Pressures occur throughout life and those <cause>pressures</cause> cause <effect>stress</effect>.\"", "causalityPhrase": "cause"}, "example 3": {"sentence": "\"The basic <effect>indicator</effect> of the health <cause>condition</cause> of trees, stands and forests is tree defoliation.\"", "causalityPhrase": "of"}, "example 4": {"sentence": "OBJECTIVE: We report a patient who developed neutropenia on <cause>clozapine</cause>, but behind the cell count decrease showed to be a <effect>diurnal variation of the white blood cells</effect> (WBC).", "causalityPhrase": "on"}, "example 5": {"

out:  {"cause": "clogging of drains", "effect": "flooding", "causality_phrase": "has been caused by"}
139
fewshot_index : [555, 556, 325, 523, 591, 518, 514, 554, 552, 557]
fewshot_examples {"example 1": {"sentence": "\"The weekend <cause>snowstorm</cause> caused <effect>delays</effect> that had some passengers still stuck in Washington, DC on Monday night.\"", "causalityPhrase": "caused"}, "example 2": {"sentence": "\"Heavy <cause>thunderstorms</cause> caused <effect>cancellation</effect> of races later in the afternoon preventing the Masters Mens fours and Masters Womens and Womens eights from racing.\"", "causalityPhrase": "caused"}, "example 3": {"sentence": "\"92% of the CO2 <effect>emissions</effect> are caused by the <cause>lorries</cause>, but this is not fair so a different distribution method is needed.\"", "causalityPhrase": "are caused by"}, "example 4": {"sentence": "\"Frustrations, <cause>threats</cause>, and conflicts cause <effect>stress</effect>.\"", "causalityPhrase":

In [10]:
len(filtered_rows)

142

In [11]:
#filtered_rows = filtered_rows.drop('GPT_causality_label', axis = 1)
#filtered_rows = filtered_rows.drop('patternmatcher_label', axis = 1)
len(fewshotfound_inputsentence_index)

142

In [12]:
filtered_rows['cause'] = cause_list
filtered_rows['effect'] = effect_list
filtered_rows['causality_phrase'] = causality_phrase_list
filtered_rows

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sentence,cause_t,effect_t,embedings,cause,effect,causality_phrase
0,107,158,158,158,158,"Finally, Slone's fear of AIDS and the mental...",stab,fear,"[-0.00929092988371849, -0.00021748989820480347...",needle stab,fear of AIDS and the mental distress,were caused by
1,108,159,159,159,159,The suffering caused by the embargo is som...,embargo,suffering,"[-0.009257652796804905, -0.031496670097112656,...",embargo,suffering,caused by
2,109,160,160,160,160,Sorace was unaware that her anger was caused...,abuse,anger,"[0.01198310311883688, -0.010999979451298714, 0...",abuse,anger,was caused by
3,110,161,161,161,161,The idea of Roman expansion into North Africa ...,power,fear,"[0.017029738053679466, -0.010211342945694923, ...",fear and jealousy,great economic power of Carthage,caused by
4,111,162,162,162,162,Tides are caused by the gravitational pull ...,gravitational pull,Tides,"[0.010173842310905457, -0.015142759308218956, ...","gravitational pull of the moon and sun, and th...",tides,are caused by
...,...,...,...,...,...,...,...,...,...,...,...,...
137,244,321,321,321,321,"The winds are caused by moisture , originat...",moisture,winds,"[-0.0030969225335866213, -0.015087406150996685...",moisture,winds,are caused by
138,245,322,322,322,322,The drag caused by the Earth's atmosphere ...,atmosphere,drag,"[-0.00888124480843544, -0.003990810364484787, ...",Earth's atmosphere,drag,caused by
139,246,323,323,323,323,Most of the flooding has been caused by the ...,clogging,flooding,"[-0.01667359657585621, 0.0030442047864198685, ...",clogging of drains,flooding,has been caused by
140,247,324,324,324,324,The dips are caused by the thermal diffuse o...,y-rays,dips,"[-0.018423471599817276, -0.0059008412063121796...",thermal diffuse of y-rays,dips,are caused by


In [13]:
def get_score(true_phrase, pred_phrase ):
    scores=[]
    p = true_phrase
    window_size = len(p)

        # Slide through input_sentence with a window size of 8
    if (pred_phrase) =="":
        print('no pred_phrase')
        return 0
    else: 
        for i in range(len(pred_phrase) - window_size + 1):
            window = pred_phrase[i:i + window_size]
            score = fuzz.ratio(window, p)
            scores.append(score)
        if len(scores) ==0:
            print('prediction length lower', pred_phrase, 'p:', p)
            return 0
        else:
            return max(scores)

In [14]:
#get accuracy
from fuzzywuzzy import fuzz

def get_accuracy(filtered_rows):
    prediction= []
    e_prediction=[]
    c_prediction=[]
    for i in range(len(filtered_rows)):
        #print(i)
        cause_pred = filtered_rows['cause'][i].lower()
        cause_t = filtered_rows['cause_t'][i].lower()
        
        effect_pred = filtered_rows['effect'][i].lower()
        effect_t = filtered_rows['effect_t'][i].lower()

        # Slide through input_sentence with a window size of 8
        #print('effect_pred',  effect_pred)
        #print('effect_t',  effect_t)


        score = get_score(cause_t, cause_pred)################ non -medcause
        #score = fuzz.ratio(cause_pred, cause_t)################medcause
        if score >90:################90 non -medcause
            pred_c = 1
            c_prediction.append(pred_c)
        else:
            pred_c = 0
            c_prediction.append(pred_c)
            
        score = get_score(effect_t, effect_pred)################non - medcause
        #score = fuzz.ratio(effect_pred, effect_t)################medcause

        if score >90: ################90 non -medcause
            pred_e = 1
            e_prediction.append(pred_e)
        else:
            pred_e = 0
            e_prediction.append(pred_e)

        
        if ((pred_e ==1) & (pred_c ==  1)):
            prediction.append(1)
        else:
            prediction.append(0)
            
    

    return prediction, c_prediction, e_prediction


In [15]:
prediction, c_prediction, e_prediction = get_accuracy(filtered_rows)

prediction length lower ulcer p: indigestion
prediction length lower some p: hormone
prediction length lower slaanesh p: suffering
prediction length lower rope p: suffocation
prediction length lower becky p: company
no pred_phrase
no pred_phrase
prediction length lower disturbance p: overpopulation


In [16]:
sum(prediction)/len(filtered_rows)

0.8802816901408451

In [16]:
sum(c_prediction)/len(filtered_rows)

0.8907970057243505

In [17]:
sum(e_prediction)/len(filtered_rows)

0.8793483047115808

In [18]:
filtered_rows['predictions'] = prediction

In [19]:
filtered_rows

,Unnamed: 0.1,Unnamed: 0,sentence,cause_t,effect_t,embedings,cause,effect,causality_phrase,predictions
0,0,0,Squamous-cell carcinoma arising in a basal-cel...,5-fluorouracil,Squamous-cell carcinoma,"[-0.010072928853332996, 0.0002307597897015512,...",5-fluorouracil,Squamous-cell carcinoma arising in a basal-cel...,treated with,1
1,1,1,Three cases of acute renal toxicity in patient...,5-fluorouracil,acute renal toxicity,"[0.003660476766526699, -0.006745004560798407, ...",long-term therapy with mitomycin C and 5-fluor...,acute renal toxicity,in patients receiving,1
2,2,2,CONCLUSIONS: Although budesonide may be benefi...,budesonide,contact dermatitis,"[-0.016381533816456795, 0.014267788268625736, ...",its anti-inflammatory effects,budesonide may be beneficial,because of,0
3,3,3,Hemodialysis should be performed for rapid rev...,mannitol,ARF,"[-0.009250188246369362, 0.008307494223117828, ...",mannitol,ARF,induced,1
4,4,4,"After the chlorambucil was discontinued, the w...",chlorambucil,AML,"[0.0037177789490669966, -0.024612924084067345,...",chlorambucil,wbc count began to slowly rise and the patient...,was discontinued,1
...,...,...,...,...,...,...,...,...,...,...
2266,2266,2266,We report here on a heretofore undescribed res...,fludarabine phosphate,respiratory syncytial virus (RSV) infection,"[-0.012739364989101887, -0.0047148303128778934...",fludarabine phosphate,respiratory syncytial virus (RSV) infection,was treated with,1
2267,2267,2267,We describe the first documented case of azath...,azathioprine,myelosuppression,"[-0.0030996142886579037, -0.015832170844078064...",thiopurine methyltransferase deficiency,azathioprine-induced severe myelosuppression,due to,0
2268,2268,2268,We report a case of biopsy-proven acute tubulo...,flurbiprofen,acute tubulointerstitial nephritis,"[0.004250906873494387, -0.0017411339795216918,...","a second course of flurbiprofen, a nonsteroida...",biopsy-proven acute tubulointerstitial nephritis,associated with,1
2269,2269,2269,Onset of male gynaecomastia in a patient treat...,sunitinib,gynaecomastia,"[-0.024055609479546547, -0.02433030679821968, ...",sunitinib,onset of male gynaecomastia,treated with,1


In [20]:
#filtered_rows.to_csv('data/out/RAG_C_E_CP_Alzeimers.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/semeval/fewshot_sem_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_

#filtered_rows.to_csv('data/out/RAG_causality/CE_extracts/randomshotCOMB_ADE_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/task2/GPT4/rand_ADE_CE_extract.csv')#zero,rand,RAG|sem,ADE,SCITE


In [21]:
#filtered_rows= filtered_rows[filtered_rows['predictions'] == 0]
#filtered_rows.to_csv('data/out/RAG_causality/semeval/zero_sem_RAG_C_E_pred0.csv')#data/out/RAG_causality/semeval/semeval_
